In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
import datetime as dt
sns.set_style('darkgrid')
import yfinance as yf

sp500_tickers = [
"WMT", "AMZN", "AAPL", "CVS", "TSLA", "GOOGL", "META", "JPM", "COST", "KR",
"WBA", "TGT", "UPS", "CNC", "CI", "MSFT", "VZ", "IBM", "UNH", "XOM",
"MCK", "CVX", "CAH", "HD", "WBA", "MPC", "COR", "F", "C", "DELL",
"GM", "NVDA", "PFE", "PG", "CMCSA", "JNJ", "WFC", "BAC", "ABBV", "SBUX",
"CSCO", "T", "PEP", "INTC", "DIS", "BA", "GS", "MS", "HON", "CRM"
]

# Get the data for tickers from yahoo finance

stock_data = yf.download(sp500_tickers,'2023-1-1','2025-10-1', auto_adjust=True)['Close']
SP500 = yf.download('^GSPC','2023-1-1','2025-10-1', auto_adjust=True)['Close']


[*********************100%***********************]  49 of 49 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
#Version with NaN values for weekends/holidays.
# Create a date range including weekends
all_days = pd.date_range(start=stock_data.index.min(), end=stock_data.index.max(), freq='D')

# Reindex the DataFrame with this date range
reindexed_stock_data = stock_data.reindex(all_days)
reindexed_SP500 = SP500.reindex(all_days)

#Function to test if a date is a trading day
def is_trading_day(date):
    return date in stock_data.index

is_trading_day('2025-1-1')

def next_trading_day(day):
    date = dt.datetime.strptime(day, '%Y-%m-%d')
    next_day = date + dt.timedelta(1)
    max_lookahead = 365  # prevent infinite loop going 1 year ahead
    count = 0
    while not is_trading_day(next_day):
        next_day += dt.timedelta(1)
        count += 1
        if count > max_lookahead:
            raise ValueError("No trading day found within 1 year from start date")
    return next_day.strftime('%Y-%m-%d')

next_trading_day('2025-1-1')


'2025-01-02'

In [11]:
#log returns
log_returns = np.log(stock_data/stock_data.shift(1)).dropna()
SP_log_returns = np.log(SP500/SP500.shift(1)).dropna()
#Percentage change in stock returns day-to-day (in lieu of trading day variance)
SP_pct_change = SP500.pct_change().dropna()
pct_change = stock_data.pct_change().dropna()
#variance of stock returns over each week in the period (5 trading days) 
SP_weekly_variance = SP_log_returns.rolling(window=5).var().dropna()
weekly_variance = log_returns.rolling(window=5).var().dropna()
#variance of stock returns over each month in the period (21 trading days)
SP_monthly_variance = SP_log_returns.rolling(window=21).var().dropna()
monthly_variance = log_returns.rolling(window=21).var().dropna()
#variance of stock returns over each 2 weeks in the period (10 trading days)
SP_biweekly_variance = SP_log_returns.rolling(window=10).var().dropna()
biweekly_variance = log_returns.rolling(window=10).var().dropna()


#super averages
#average percent change in a single day
Av_SP_pct_change = SP_pct_change.mean()
av_pct_change = pct_change.mean()
#average weekly variance
av_weekly_variance = weekly_variance.mean()
Av_SP_weekly_variance = SP_weekly_variance.mean()
#average monthly variance
av_monthly_variance = monthly_variance.mean()
Av_SP_monthly_variance = SP_monthly_variance.mean()
#average biweekly variance
av_biweekly_variance = biweekly_variance.mean()
Av_SP_biweekly_variance = SP_biweekly_variance.mean()

#Note: variance windows are in trading days, not calendar days. Moreover, these are backwards looking rolling variances.

C:\Users\Thisu\AppData\Local\Temp\ipykernel_22040\1458790297.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  pct_change = stock_data.pct_change().dropna()


Notes: Need to account for weekend social media activity